In [ ]:
import unicodecsv
from datetime import datetime as dt

enrollment_filename= 'D:\Github\DAWithPython\pyScript\data\enrollments.csv'
dailyEngagement_filename= 'D:\Github\DAWithPython\pyScript\data\daily_engagement.csv'
projectsubmissions_filename= 'D:\Github\DAWithPython\pyScript\data\project_submissions.csv'

#read file
def readCsv(filename):
    with open(filename,'rb') as f:
        reader = unicodecsv.DictReader(f)
        return list(reader)

enrollments=readCsv(enrollment_filename)
dailyEngagements=readCsv(dailyEngagement_filename)
projectsubmissions=readCsv(projectsubmissions_filename)

#format data
def parse_int(i):
    if i==None or i=='':
        return None
    else:
        return int(i)

def parse_date(date):
    if date==None or date=='':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')

def parse_float(f):
    if f==None or f=='':
        return 0.0
    else:
        return float(f)

def getUniqueSet(di):
    uniqueSet=set()    
    for d in di:
        uniqueSet.add(d['account_key'])
    return uniqueSet

for enrollment in enrollments:
    enrollment['days_to_cancel'] = parse_int(enrollment['days_to_cancel'])
    enrollment['is_canceled'] = enrollment['is_canceled']=='True'
    enrollment['join_date'] = parse_date(enrollment['join_date'])
    enrollment['is_udacity'] = enrollment['is_udacity']=='True'
    enrollment['cancel_date'] = parse_date(enrollment['cancel_date'])

for dailyEngagement in dailyEngagements:
    dailyEngagement['lessons_completed'] = parse_float(dailyEngagement['lessons_completed'])
    dailyEngagement['projects_completed'] = parse_float(dailyEngagement['projects_completed'])
    dailyEngagement['total_minutes_visited'] = parse_float(dailyEngagement['total_minutes_visited'])
    dailyEngagement['utc_date'] = parse_date(dailyEngagement['utc_date'])
    dailyEngagement['num_courses_visited'] = int(parse_float(dailyEngagement['num_courses_visited']))
    dailyEngagement['account_key'] = dailyEngagement['acct']
    del[dailyEngagement['acct']]

for projectsubmission in projectsubmissions:
    projectsubmission['completion_date'] = parse_date(projectsubmission['completion_date'])
    projectsubmission['creation_date'] = parse_date(projectsubmission['creation_date'])
    
#get unique data
##to get unique values in the dictionary
unique_enrolled_students = getUniqueSet(enrollments)
unique_daily_engagements = getUniqueSet(dailyEngagements)
unique_project_submissions = getUniqueSet(projectsubmissions)
        
##########THIS IS NEW HERE###################
#to remove ud accounts data from all the lists
ud_Acct=set()
for enrollment in enrollments:
    if enrollment['is_udacity']:
        ud_Acct.add(enrollment['account_key'])

def removeUdAcct(di):
    nonud=[]    
    for d in di:
        if d['account_key'] not in ud_Acct:
           nonud.append(d)
    return nonud

nonud_enroll=removeUdAcct(enrollments)
nonud_engagement=removeUdAcct(dailyEngagements)
nonud_projSubmission=removeUdAcct(projectsubmissions)

##to get the dictionary of students who havent cancelled or have cancelled atleast 8 days after joining

In [20]:
paid_students=dict()
for stud in nonud_enroll:
    if not stud['is_canceled'] or stud['days_to_cancel']>7:
        account_key=stud['account_key']
        date = stud['join_date']
        if stud['account_key'] not in paid_students or date>paid_students[account_key]:
            paid_students[account_key]=date

In [44]:
print(paid_students['777'])
print(len(paid_students))

2015-06-08 00:00:00
995


In [28]:
def remove_cancelled_reg(data):
    paid_data=[]
    for d in data:
        if d['account_key'] in paid_students:
            paid_data.append(d)
    return paid_data

In [40]:
paid_enrollments = remove_cancelled_reg(nonud_enroll)
paid_engagement = remove_cancelled_reg(nonud_engagement)
paid_projSubmission = remove_cancelled_reg(nonud_projSubmission)
print (len(paid_enrollments))
print (len(paid_engagement))
print (len(paid_projSubmission))

1293
134549
3618


In [34]:
def within_one_week(join_date, engagement_date):
    time_delta = engagement_date - join_date
    return time_delta.days < 7

paid_engagement_in_first_week = []
for engagement_record in paid_engagement:
    account_key = engagement_record['account_key']
    join_date = paid_students[account_key]
    engagement_record_date = engagement_record['utc_date']

    if within_one_week(join_date, engagement_record_date):
        paid_engagement_in_first_week.append(engagement_record)

In [35]:
len(paid_engagement_in_first_week)

17210